In [11]:
from transformers import AutoModel,AutoTokenizer,BertTokenizerFast,AutoModelForSequenceClassification
import torch
from torch import nn
import pandas as pd
from datasets import load_dataset
import numpy as np
from torch.utils.data import Dataset,DataLoader
import random
from transformers import AdamW
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score , accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#加载模型
model = AutoModel.from_pretrained('hfl/chinese-macbert-base',cache_dir = './model') # 
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

c:\Users\DELL\anaconda3\envs\llama3conda\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\code\vsc\Transformerc\MacBERT\macbert_end\model\models--hfl--chinese-macbert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

c:\Users\DELL\anaconda3\envs\llama3conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [36]:
#冻结模型参数
for pa in model.parameters():
    pa.requires_grad = False

In [3]:
data = load_dataset('./exercise_contest')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
# itos and stoi
name_to_num = {}
num_to_name =  {}
count = 0
for i in data['train']:
    for j in i['meta']['accusation']:
        if(j not in name_to_num.keys()):
            name_to_num[j]=count
            count+=1
for key, value in name_to_num.items():
    num_to_name[value] = key

In [13]:
# 处理数据集不平衡 修改每种罪名数据大概要加载多少条
name_per_label_number = {k:0 for k in name_to_num.keys()}
for i in data['train']:
    for single_name in i['meta']['accusation']:
        name_per_label_number[single_name] +=1
name_per_label_number
sample_number_per_label = {k:0 for k in name_to_num.keys()}
for k,v in name_per_label_number.items():
    v = v / 7
    if v < 30:
        sample_number_per_label[k] = 30
    else:
        number = math.floor(v / 2 ) + 30
        sample_number_per_label[k] = number

19276

In [8]:
class Mydataset(Dataset):
    def __init__(self,dataset_type:str,tokenizer:BertTokenizerFast=tokenizer):
        self.sample_list = []
        self.tokenizer = tokenizer
        # 加载对应数据集
        data_type = data[dataset_type].shuffle(seed = 3)
        # 对训练数据进行采样 按照罪名应该采样多少
        if data_type == 'train':
            current_number_per_label = {k:0 for k in name_to_num.keys()}
            for i in data_type:
                current_data_i = random.choice(i['meta']['accusation'])
                if current_number_per_label[current_data_i] > sample_number_per_label[current_data_i]:
                    continue
                input_data = str({'fact':i['fact'],'life_imprisonment':i['meta']['term_of_imprisonment']['life_imprisonment'],'death_penalty': i['meta']['term_of_imprisonment']['death_penalty'],'imprisonment': i['meta']['term_of_imprisonment']['imprisonment'],'criminals':i['meta']['criminals'],'relevant_articles':i['meta']['relevant_articles']})
                label = list(map(lambda x: name_to_num[x],i['meta']['accusation']))
                self.sample_list.append((input_data,label))
                current_number_per_label[current_data_i] += 1
        else:
            # 普通采样
            count = 0
            for i in data_type:
                if len(i['fact']) < 328:
                    input_data = str({'fact':i['fact'],'life_imprisonment':i['meta']['term_of_imprisonment']['life_imprisonment'],'death_penalty': i['meta']['term_of_imprisonment']['death_penalty'],'imprisonment': i['meta']['term_of_imprisonment']['imprisonment'],'criminals':i['meta']['criminals'],'relevant_articles':i['meta']['relevant_articles']})
                    label = list(map(lambda x: name_to_num[x],i['meta']['accusation']))
                    self.sample_list.append((input_data,label))
                    count += 1
                    if count >7000:
                        break
    def __getitem__(self,index):
        input_data,label = self.sample_list[index]
        label_onehot = torch.zeros(len(name_to_num))
        for label_i in label:
            label_onehot[label_i] = 1
        input_tokenized = self.tokenizer(text=input_data,padding='max_length',max_length=512,truncation=True,return_tensors='pt')
        return {
            'input_data':input_data,
            'input_ids':input_tokenized['input_ids'].squeeze(),
            'attention_mask':input_tokenized['attention_mask'].squeeze(),
            'label':label_onehot
        }
    def __len__(self):
        return len(self.sample_list)


In [9]:
# 加了头后的模型
class macbert(nn.Module):
    def  __init__(self):
        super(macbert,self).__init__()
        self.bert = model
        # self.dropout = nn.Dropout(0.1)
        self.classifier = classifyner()
    def forward(self, input_ids,attention_mask):
        output_1 = self.bert(input_ids,attention_mask)
        output_2 = output_1['last_hidden_state']
        output_2 = output_2.mean(dim=1)
        logits = self.classifier(output_2)
        return logits
class classifyner(nn.Module):
    def __init__(self):
        super(classifyner,self).__init__()
        self.fc_1 = nn.Linear(768, 768)
        self.gule = nn.GELU()
        self.fc_2 = nn.Linear(768, 202)

    def forward(self, x):
        x = self.fc_1(x)
        x = self.gule(x)
        x = self.fc_2(x)
        return x
    

In [23]:
data_train = Mydataset('train',tokenizer)
data_train = DataLoader(data_train,batch_size=4,shuffle=True)

In [39]:
# 训练
device = torch.device('cuda:0')
mo = macbert().to(device)
mo.load_state_dict(torch.load('model2.pth'))
optimizer = AdamW(filter(lambda p: p.requires_grad,mo.parameters()), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
epochs = 3
for epoch in range(epochs):
    mo.train()
    train_tqdm = tqdm(data_train)
    for batch in train_tqdm:
        input_ids ,attention_mask, label = batch['input_ids'].to(device),batch['attention_mask'].to(device),batch['label'].to(device)
        optimizer.zero_grad()
        output = mo(input_ids,attention_mask)
        loss = criterion(output,label)
        train_tqdm.set_postfix(loss = loss.item())
        loss.backward()
        optimizer.step()
    torch.save(mo.state_dict(),f'model_sample{epoch+1}.pth')

C:\Users\DELL\AppData\Local\Temp\ipykernel_30628\2440931227.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mo.load_state_dict(torch.load('model2.pth'))
100%|██████████|

In [40]:
# 验证模型的准确率召回率和正确率

data_valid = Mydataset('validation',tokenizer)
data_valid = DataLoader(data_valid,batch_size=4,shuffle=True)

model_valid = macbert().to(device)
model_valid.load_state_dict(torch.load('model_sample3.pth'))
sig = nn.Sigmoid()

pred_list = []
label_list = []
data_tqdm = tqdm(data_valid)

model_valid.eval()

for t in tqdm(data_tqdm):
    input_ids ,attention_mask, label = t['input_ids'].to(device),t['attention_mask'].to(device),t['label'].to(device)
    output = model_valid(input_ids,attention_mask)
    pred = sig(output)
    for i in zip(pred,label):
        pred_list.append((i[0]>0.9).to(torch.int))
        label_list.append(i[1])
pred_list = [i.cpu().detach().numpy() for i in pred_list]
label_list = [i.cpu().detach().numpy() for i in label_list]

p_score = precision_score(pred_list,label_list,average='samples')
r_score = recall_score(pred_list,label_list,average='samples') 
a_score = accuracy_score(pred_list,label_list)
print(p_score,r_score,a_score)
# 0.8730181402656764 0.8825405894396038 0.8598771604056563

C:\Users\DELL\AppData\Local\Temp\ipykernel_30628\81377433.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_valid.load_state_dict(torch.load('model_sample3.pth'))
10

0.8730181402656764 0.8825405894396038 0.8598771604056563


c:\Users\DELL\anaconda3\envs\llama3conda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# 模型测试输入输出

model_test = macbert().to(device)
model_test.load_state_dict(torch.load('model_sample3.pth'))

sig = nn.Sigmoid()
def get_pred(input_data):
    model_test.eval()
    input_tokenized = tokenizer(text=input_data,padding='max_length',max_length=512,truncation=True,return_tensors='pt')
    input_ids = input_tokenized['input_ids'].to(device)
    attention_mask = input_tokenized['attention_mask'].to(device)
    pred = []
    with torch.no_grad():
        output = model_test(input_ids,attention_mask)
        
        metric = (sig(output) > 0.9).to(torch.int).squeeze().cpu().numpy()
        output = [i for i,v in enumerate(metric) if v==1]
        for i in output:
            pred.append(num_to_name[i])
    return pred

C:\Users\DELL\AppData\Local\Temp\ipykernel_23804\3753101824.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_test.load_state_dict(torch.load('model_sample3.pth'))


In [ ]:
input = '''"{'fact': '永嘉县人民检察院指控，2014年4月至2015年9月期间，被告人李某在未取得医生执业资格的情况下，伙同他人在温州市瑞安市汀田街道联盟村、瑞安市塘下等地用便携式b超机先后为张某、刘某、缪某某、雷某某、兰某等人进行非医学需要的胎儿性别鉴定，共7人次以上，并导致他人将胎儿引产。其中，被告人李某在2015年8月25日为缪某某、雷某某进行非医学需要的胎儿性别鉴定时被温州经济技术开发区民政卫生和计划生育局查获，并被罚款人民币1万元。案发后，被告人李某于2015年11月5日于温州市瑞安市汀田街道联盟村盟兴巷26号被民警抓获。', 'life_imprisonment': False, 'death_penalty': False, 'imprisonment': 9, 'criminals': ['李某'], 'relevant_articles': [336]}"'''
get_pred(input)[0]

c:\Users\DELL\anaconda3\envs\llama3conda\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'非法行医'